In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import os

# Define RRMSE and ARRSME scoring functions
def rrmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mean_y = np.mean(y_true)
    return rmse / mean_y

def arrmse(y_true, y_pred):
    n_targets = y_true.shape[1]
    rrmse_scores = []
    for i in range(n_targets):
        rrmse_scores.append(rrmse(y_true[:, i], y_pred[:, i]))
    return np.mean(rrmse_scores)

# Enable GPU for XGBoost training if available
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Ensure only GPU 0 is used (or adjust if multiple GPUs)

def train_xgb_on_gpu(x_train, y_train, x_test, y_test, target_cols):
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_test, label=y_test)
    params = {
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'eval_metric': 'rmse'
    }
    num_round = 100
    # Train one model per target column and predict
    y_pred = np.zeros(y_test.shape)
    for i, target in enumerate(target_cols):
        bst = xgb.train(params, xgb.DMatrix(x_train, label=y_train[:, i]), num_round)
        y_pred[:, i] = bst.predict(xgb.DMatrix(x_test))
    return y_pred

# Example usage after splits are done:
# y_pred = train_xgb_on_gpu(x_train.values, y_train.values, x_test.values, y_test.values, list(y_train.columns))
# score = arrmse(y_test.values, y_pred)
# print('ARRMSE:', score)
